In [3]:
import numpy as np
import pandas as pd
from Preprocessing import *
import torch.nn as nn
import torch

In [21]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

In [22]:
noise = create_noise(64, 100, 'normal')

In [23]:
noise.shape

torch.Size([64, 100])

In [24]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim, output_shape):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.output_shape = output_shape

        # Define your layers here
        self.fc = nn.Linear(latent_dim, 1 * 105 * 8)
        self.conv_transpose = nn.ConvTranspose2d(1, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, z):
        # Reshape the input
        x = self.fc(z)
        x = x.view(-1, 1, 105, 8)

        # Apply convolutional transpose layer
        x = self.conv_transpose(x)

        return x

# Example usage
latent_dim = 100
output_shape = (64, 1, 105, 8)

# Create an instance of the generator
generator = Generator(latent_dim, output_shape)

# Generate a random input
z = torch.randn((64, latent_dim))

# Forward pass to generate an output
output = generator(noise)

print("Generator Output Shape:", output.shape)


Generator Output Shape: torch.Size([64, 1, 105, 8])


In [5]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, noise_dim, output_shape):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim

        # Define the layers of your generator
        self.fc = nn.Linear(noise_dim, 105 * 8)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)

        self.output_shape = output_shape

    def forward(self, z):
        # Reshape the noise vector
        z = self.fc(z)
        z = z.view(z.size(0), 1, 105, 8)

        # Upsample and generate the output
        z = self.conv1(z)
        z = self.bn1(z)
        z = self.relu(z)
        z = self.conv2(z)

        return z

# Example usage
noise_dim = 100  # Adjust the noise dimension as needed
output_shape = (1, 105, 8)
batch_size = 64  # Adjust the batch size as needed

# Create an instance of the generator
generator = Generator(noise_dim, output_shape)

# Generate random noise
noise = torch.randn(batch_size, noise_dim)

# Generate fake data
fake_data = generator(noise)

# Check the shape of the generated data
print(fake_data.shape)


torch.Size([64, 1, 105, 8])


In [1]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, noise_dim, word_embedding_dim, output_shape):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.word_embedding_dim = word_embedding_dim

        # Define the layers of your generator
        self.fc_noise = nn.Linear(noise_dim, 105 * 8)
        self.fc_word_embedding = nn.Linear(word_embedding_dim, 105 * 8)
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)

        self.output_shape = output_shape

    def forward(self, noise, word_embedding):
        # Process noise
        noise = self.fc_noise(noise)
        noise = noise.view(noise.size(0), 1, 105, 8)

        # Process word embedding
        word_embedding = self.fc_word_embedding(word_embedding)
        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 8)

        # Concatenate noise and word embedding
        combined_input = torch.cat([noise, word_embedding], dim=1)

        # Upsample and generate the output
        z = self.conv1(combined_input)
        z = self.bn1(z)
        z = self.relu(z)
        z = self.conv2(z)

        return z

# Example usage
noise_dim = 100  # Adjust the noise dimension as needed
word_embedding_dim = 50  # Adjust the word embedding dimension as needed
output_shape = (1, 105, 8)
batch_size = 16  # Adjust the batch size as needed

# Create an instance of the generator
generator = Generator(noise_dim, word_embedding_dim, output_shape)

# Generate random noise and word embedding
noise = torch.randn(batch_size, noise_dim)
word_embedding = torch.randn(batch_size, word_embedding_dim)

# Generate fake data
fake_data = generator(noise, word_embedding)

# Check the shape of the generated data
print(fake_data.shape)


torch.Size([16, 1, 105, 8])


In [9]:
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize  # You might need to install nltk: pip install nltk

# Example list of words
word_list = ["example", "list", "of", "words"]

# Tokenize the words
tokenized_words = [word_tokenize(word) for word in word_list]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_words, vector_size=100, window=5, min_count=1, workers=4)

# Get the word embeddings
word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}

# Example usage
for word, embedding in word_embeddings.items():
    print(f"Word: {word}, Embedding: {embedding}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Word: words, Embedding: [-5.3622725e-04  2.3643136e-04  5.1033497e-03  9.0092728e-03
 -9.3029495e-03 -7.1168090e-03  6.4588725e-03  8.9729885e-03
 -5.0154282e-03 -3.7633716e-03  7.3805046e-03 -1.5334714e-03
 -4.5366134e-03  6.5540518e-03 -4.8601604e-03 -1.8160177e-03
  2.8765798e-03  9.9187379e-04 -8.2852151e-03 -9.4488179e-03
  7.3117660e-03  5.0702621e-03  6.7576934e-03  7.6286553e-04
  6.3508903e-03 -3.4053659e-03 -9.4640139e-04  5.7685734e-03
 -7.5216377e-03 -3.9361035e-03 -7.5115822e-03 -9.3004224e-04
  9.5381187e-03 -7.3191668e-03 -2.3337686e-03 -1.9377411e-03
  8.0774371e-03 -5.9308959e-03  4.5162440e-05 -4.7537340e-03
 -9.6035507e-03  5.0072931e-03 -8.7595852e-03 -4.3918253e-03
 -3.5099984e-05 -2.9618145e-04 -7.6612402e-03  9.6147433e-03
  4.9820580e-03  9.2331432e-03 -8.1579173e-03  4.4957981e-03
 -4.1370760e-03  8.2453608e-04  8.4986202e-03 -4.4621765e-03
  4.5175003e-03 -6.7869602e-03 -3.5484887e-03  9.3985079e-03
 -1.5776526e-03  3.2137157e-04 -4.1406299e-03 -7.6826881e-03


In [39]:
def data_structure(raw):
    sampling_freq = raw.info['sfreq']
    start_end_secs = np.array([0,4850]) # Fro this dataset data start at 0.000 and end at 4840.166 secs
    start_sample, stop_sample = (start_end_secs * sampling_freq).astype(int)
    df = raw.to_data_frame(picks=['all'], start=start_sample, stop=stop_sample)
    # then save using df.to_csv(...), df.to_hdf(...), etc

    df = df.drop(['time', "STI 014"], axis=1)
    return df

In [40]:
def create_rolling_window(data, window_size=5):
    """
    This function creates a rolling window of a given size from a data frame.
    It assigns the majority label to the window.

    Creator : Niall

    Args:
        data: Dataframe to be windowed
        window_size: The size of the window to be created

    Returns:
        windows: A list of dataframes containing the windows
        labels: A list of labels corresponding to the windows

    """

    windows = []
    labels = []
    data = data.to_numpy()
    for i in range(len(data) - window_size + 1):
        window_data = data[i:i + window_size]
        label = window_data[:, -1][0]  # Assign majority label
        labels.append(label)
        window_data = window_data[:, :-1]
        windows.append(window_data)

    return windows, labels

In [41]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return sample

In [42]:
raw = import_data(r"C:\Users\niall\Desktop\PhD\OpenMIIR-RawEEG_v1\P04-raw.fif")

Opening raw data file C:\Users\niall\Desktop\PhD\OpenMIIR-RawEEG_v1\P04-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2480032 =      0.000 ...  4843.812 secs
Ready.


In [43]:
df = data_structure(raw)

In [44]:
windows, labels = create_rolling_window(df, window_size=5)

In [45]:
data_list = windows[:200]  # Replace with your actual data

# Convert the list of arrays to PyTorch tensors
data_tensors = [torch.Tensor(array) for array in data_list]

# Stack the tensors to create a 3D tensor (200x5x67)
data_tensor = torch.stack(data_tensors, dim=0)

In [46]:
from torch.utils.data import DataLoader

batch_size = 32  # Set your desired batch size
shuffle = True   # You can set this to True if you want to shuffle the data

dataset = CustomDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [13]:
for batch in dataloader:
    print(batch.shape)
    # Do stuff with the batch

torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([32, 5, 67])
torch.Size([8, 5, 67])


In [15]:
def make_discriminator_network(
        input_size,
        num_hidden_layers=1,
        num_hidden_units=100,
        num_output_units=1
):
    model = nn.Sequential()
    for i in range(num_hidden_layers):
        model.add_module(
            f'fc_d{i}',
            nn.Linear(input_size, num_hidden_units, bias=False)
        )
        model.add_module(f'relu_d{i}', nn.LeakyReLU())
        model.add_module('dropout', nn.Dropout(p=0.5))
        input_size = num_hidden_units
    model.add_module(f'fc_d{num_hidden_layers}',
                     nn.Linear(input_size, num_output_units))
    model.add_module('sigmoid', nn.Sigmoid())
    return model

In [157]:
disc_model = make_discriminator_network(
    input_size=335,
    num_hidden_layers=1,
    num_hidden_units=32
)

In [158]:
print(disc_model)

Sequential(
  (fc_d0): Linear(in_features=335, out_features=32, bias=False)
  (relu_d0): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_d1): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [159]:
input_real = next(iter(dataloader))
input_real = input_real.view(batch_size, -1)

In [160]:
input_real.shape

torch.Size([32, 335])

In [164]:
torch.manual_seed(1)
d_proba_real = disc_model(input_real)

In [165]:
d_proba_real

tensor([[1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<SigmoidBackward0>)

In [166]:
d_proba_real.shape

torch.Size([32, 1])